# Лабораторная работа 4. Ранжирование.



Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 9 баллов. Сдавать задание после указанного в lk срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).



## Знакомство с данными

### Ранжирование организаций по пользовательскому запросу

Что мы обычно делаем, когда нам нужно найти определённое место, но не знаем его местоположения? Используем поиск на картах.

В этой лабораторной работе вам будет необходимо построить небольшую поисковую систему, позволяющую отранжировать организации по запросу пользователя.

Для обучения вам даны 2000 запросов и более 13 тысяч найденных по ним организаций. Для каждой пары "запрос — организация" была посчитана релевантность, по которой и происходит ранжирование.

**(2 балла) Задание 1.** Загрузите [данные](https://disk.yandex.ru/d/Bf3P4H8FDYe7-g) о запросах и их релевантности (*train.csv*), а также информацию об организациях (*train_org_information.json*) и рубриках (*train_rubric_information.json*)

Для дальнейших экспериментов необходимо посчитать небольшой набор базовых факторов. С использованием информации о запросах и организациях, посчитайте факторы, которые на ваш взгляд будут полезными для предсказания релевантности.

Примерами текстовых факторов могут служить:
 - кол-во слов в запросе и названии организации;
 - пословные/N-граммные пересечения слов запроса и названия организации (также можно использовать синонимы названия организации и адрес организации): кол-во слов в пересечении, [мера Жаккара](https://en.wikipedia.org/wiki/Jaccard_index) и пр.;
 - кол-во различных синонимичных названий организации (поле *names* в описании организации);
 - One-hot-encoded язык запроса.
 
По информации о географическом положении:
 - факт совпадения региона, где задавался запрос и региона организации;
 - координаты показанной области;
 - размеры показанной области;
 - меры, характеризующие близость координат организации к показанному окну: расстояние до центра области и другие.
 
Факторы, описывающие организацию:
 - one-hot-encoding фактор cтраны или региона организации (важно: не используйте one-hot-encoding факторы, в которых больше 10 значений; если в факторе слишком много значений, ограничьтесь, например, только самыми популярными категориями)
 - кол-во рабочих дней в неделе и общая продолжительность работы (поле *work_intervals* в описании организации)
 - кол-во рубрик (поле *rubrics* в описании организации)
 
![](https://miro.medium.com/max/1500/0*FwubnnoNlt6Coo9j.png)

В этом задании не нужно использовать многомерные представления текстовой информации (tfidf и прочие embeddings) и информацию о кликах (*train_clicks_information.json*). Придумывать сверхсложные факторы тоже необязательно.

Вы можете реализовать описанные выше факторы и/или придумать свои. Но зачастую такие простые признаки могут приносить наибольшую пользу.

В итоге у вас должно получиться от 15 до 50 факторов, характеризующих запрос и организацию и покрывающих основные источники данных (кроме кликов). Это наш основной датасет, который будет использоваться в экспериментах.



In [256]:
import numpy as np
import pandas as pd
from nltk import ngrams
from functools import partial
import chardet
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.preprocessing import OneHotEncoder
import json
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ndcg_score
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import types

import xgboost as xgb
import catboost
import sklearn
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

from catboost import Pool, cv
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import functools


from geopy.distance import distance
from sklearn.model_selection import train_test_split

import numpy as np
from geopy.geocoders import Nominatim

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [206]:
with open('ranking_data_shad/train_org_information.json') as f:
    org_info = json.load(f)

with open('ranking_data_shad/train_rubric_information.json') as f:
    rubric_info = json.load(f)

train = pd.read_csv('ranking_data_shad/train.csv')

In [207]:
org_features = []
geolocator = Nominatim(user_agent="geoapiExercises")
for org_id, org in tqdm(org_info.items()):
    lon, lat = org['address']['pos']['coordinates']

    rubrics_num = len(org['rubrics'])
    region_id = org['address']['geo_id']

    working_days_num = 0
    for interval in org['work_intervals']:
        if interval['day'] == 'weekdays':
            working_days_num += 5
        elif interval['day'] == 'everyday':
            working_days_num += 7
        else:
            working_days_num += 1

    region_code = org['address']['region_code']

    org_features.append({
        'org_id': org_id,
        'lat': lat,
        'lon': lon,
        'rubrics_num': rubrics_num,
        'org_region_id': region_id,
        'working_days_num': working_days_num,
        'region_code': region_code
    })

orgs_df = pd.DataFrame(org_features)
orgs_df['org_id'] = orgs_df['org_id'].astype(int)

  0%|          | 0/27697 [00:00<?, ?it/s]

In [208]:
def one_hot_encode(df, feature_column, trashhold, prefix, drop_original_column=False):
    df = df.copy()
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    df_oht = pd.DataFrame(enc.fit_transform(df[feature_column].values.reshape(-1, 1)), columns=enc.categories_)
    for column in df_oht.columns:
        if df_oht[column].mean() > trashhold:
            df[f'{prefix}{column}'] = df_oht[column]
    if drop_original_column:
        df.drop(columns=[feature_column], inplace=True)

    return df

In [209]:
orgs_df = one_hot_encode(orgs_df, 'org_region_id', 0.03, 'is_region_', drop_original_column=False)
orgs_df = one_hot_encode(orgs_df, 'region_code', 0.03, 'is_region_code_', drop_original_column=False)

In [210]:
train['query'] = train['query'].str.findall('\w+').str.join(' ').str.lower()
train['org_name'] = train['org_name'].str.findall('\w+').str.join(' ').str.lower()
train['window_center'] = train['window_center'].str.split(',').str[::-1]
train['window_size'] = train['window_size'].apply(lambda x: np.sqrt(float(x.split(',')[0])**2 + float(x.split(',')[1])**2))

In [211]:
train = pd.merge(train, orgs_df, on='org_id')
train['org_point'] = train['lat'].astype(str) + ',' + train['lon'].astype(str)

In [212]:
train['distance'] = train.parallel_apply(lambda row: distance(row['window_center'], row['org_point']).km, axis=1)
train['word_count_query'] = train['query'].str.split().str.len()
train['word_count_org'] = train['org_name'].str.split().str.len()
train['is_same_region'] = train['region'] == train['org_region_id']

In [213]:
def find_intersection(row, n):
    query_gramms = set(ngrams(row['query'].split(), n))
    org_gramms = set(ngrams(row['org_name'].split(), n))

    return len(query_gramms & org_gramms)
def find_jaccard(row, n):
    query_gramms = set(ngrams(row['query'].split(), n))
    org_gramms = set(ngrams(row['org_name'].split(), n))

    return len(query_gramms & org_gramms) / (len(org_gramms - query_gramms) + len(query_gramms - org_gramms) + 0.001)
for word_count in (1, 2, 3):
    apply_intersection = partial(find_intersection, n=word_count)
    train[f'intersection_{word_count}_gramms']= train.progress_apply(apply_intersection, axis=1)

    apply_jaccard = partial(find_jaccard, n=word_count)
    train[f'jaccard_{word_count}_gramms']= train.progress_apply(apply_jaccard, axis=1)

  0%|          | 0/29274 [00:00<?, ?it/s]

  0%|          | 0/29274 [00:00<?, ?it/s]

  0%|          | 0/29274 [00:00<?, ?it/s]

  0%|          | 0/29274 [00:00<?, ?it/s]

  0%|          | 0/29274 [00:00<?, ?it/s]

  0%|          | 0/29274 [00:00<?, ?it/s]

In [214]:
def detect_language(string):
    try:
        return detect(string)
    except Exception as e:
        return 'unk'

train['org_name_language'] = train['org_name'].parallel_apply(detect_language)
train['query_language'] = train['query'].parallel_apply(detect_language)

In [215]:
train = one_hot_encode(train, 'query_language', 0.05, 'is_query_', drop_original_column=False)
train = one_hot_encode(train, 'org_name_language', 0.05, 'is_org_name_', drop_original_column=False)

## Ранжирование

![](http://i.imgur.com/2QnD2nF.jpg)

Задачу поискового ранжирования можно описать следующим образом: имеется множество документов $d \in D$ и множество запросов $q \in Q$. Требуется оценить *степень релевантности* документа по отношению к запросу: $(q, d) \mapsto r$, относительно которой будет производиться ранжирование. Для восстановления этой зависимости используются методы машинного обучения. Обычно используется три типа:
 - признаки запроса $q$, например: мешок слов текста запроса, его длина, ...
 - документа $d$, например: значение PageRank, мешок слов, доменное имя, ...
 - пары $(q, d)$, например: число вхождений фразы из запроса $q$ в документе $d$, ...

Одна из отличительных особенностей задачи ранжирования от классических задач машинного обучения заключается в том, что качество результата зависит не от предсказанных оценок релевантности, а от порядка следования документов в рамках конкретного запроса, т.е. важно не абсолютное значение релевантности (его достаточно трудно формализовать в виде числа), а то, более или менее релевантен документ, относительно других документов.
### Подходы к решению задачи ранжирования
Существуют 3 основных подхода, различие между которыми в используемой функции потерь:
  
1. **Pointwise подход**. В этом случае рассматривается *один объект* (в случае поискового ранжирования - конкретный документ) и функция потерь считается только по нему. Любой стандартный классификатор или регрессор может решать pointwise задачу ранжирования, обучившись предсказывать значение таргета. Итоговое ранжирование получается после сортировки документов к одному запросу по предсказанию такой модели.
2. **Pairwise подход**. В рамках данной модели функция потерь вычисляется по *паре объектов*. Другими словами, функция потерь штрафует модель, если отражированная этой моделью пара документов оказалась в неправильном порядке.
3. **Listwise подход**. Этот подход использует все объекты для вычисления функции потерь, стараясь явно оптимизировать правильный порядок.

### Оценка качества

Для оценивания качества ранжирования найденных документов в поиске используются асессорские оценки. Само оценивание происходит на скрытых от обучения запросах $Queries$. Для этого традиционно используется метрика *DCG* ([Discounted Cumulative Gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)) и ее нормализованный вариант — *nDCG*, всегда принимающий значения от 0 до 1.
Для одного запроса DCG считается следующим образом:
$$ DCG = \sum_{i=1}^P\frac{(2^{rel_i} - 1)}{\log_2(i+1)}, $$

где $P$ — число документов в поисковой выдаче, $rel_i$ — релевантность (асессорская оценка) документа, находящегося на i-той позиции.

*IDCG* — идеальное (наибольшее из возможных) значение *DCG*, может быть получено путем ранжирования документов по убыванию асессорских оценок.

Итоговая формула для расчета *nDCG*:

$$nDCG = \frac{DCG}{IDCG} \in [0, 1].$$

Чтобы оценить значение *nDCG* на выборке $Queries$ ($nDCG_{Queries}$) размера $N$, необходимо усреднить значение *nDCG* по всем запросам  выборки:
$$nDCG_{Queries} = \frac{1}{N}\sum_{q \in Queries}nDCG(q).$$

Пример реализации метрик ранжирование на python можно найти [здесь](https://gist.github.com/mblondel/7337391).

В рамках нашей задачи «документом» будет являться организация.

Разбейте обучающую выборку на обучение и контроль в соотношении 70 / 30. Обратите внимание, что разбивать необходимо множество запросов, а не строчки датасета.

In [216]:
queries = train['query_id'].unique()

In [217]:
queries_train, queries_test = train_test_split(queries, test_size=0.3, random_state=42)

In [218]:
test = train[train['query_id'].isin(set(queries_test))].reset_index()
train = train[train['query_id'].isin(set(queries_train))].reset_index()

In [219]:
train.sort_values('query_id', inplace=True)
test.sort_values('query_id', inplace=True)

Далее рассмотрим несколько подходов предсказания релевантности. Для оценивания качества моделей используйте метрику nDCG на контроле. В случае подбора гиперпараметров используйте кросс-валидацию по 5 блокам, где разбиение должно быть по запросам, а не строчкам датасета.

###  Ранжируем с XGBoost и CatBoost

XGBoost имеет несколько функций потерь для решения задачи ранжирования:
1. **reg:linear** — данную функцию потерь можно использовать для решения задачи ранжирование *pointwise* подходом.
2. **rank:pairwise** — в качестве *pairwise* модели в XGBoost реализован [RankNet](http://icml.cc/2015/wp-content/uploads/2015/06/icml_ranking.pdf), в котором минимизируется гладкий функционал качества ранжирования: $$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = log(1 + e^{-M}), $$ где $ a(x) $ - функция ранжирования. Суммирование ведется по всем парам объектов, для которых определено отношение порядка, например, для пар документов, показанных по одному запросу. Таким образом функция потерь штрафует за то, что пара объектов неправильно упорядочена.
3. **rank:map, rank:ndcg** — реализация [LambdaRank](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf) для двух метрик: [MAP](https://en.wikipedia.org/wiki/Information_retrieval#Mean_average_precision) и **nDCG**. Известно, что для того, чтобы оптимизировать негладкий функционал, такой как **nDCG**,  нужно домножить градиент функционала $ Obj(a) $ на значение $\Delta NDCG_{ij} $ — изменение значения функционала качества при замене $x_i$ на $ x_j$.  Поскольку для вычисления метрик необходимы все объекты выборки, то эти две ранжирующие функции потерь являются представителями класса *listwise* моделей.

Реализованные в CatBoost ранжирующие функции потерь можной найти [здесь](https://catboost.ai/docs/concepts/loss-functions-ranking.html#groupwise-metrics).

**(3 балла) Задание 2.** Попробуйте различные функции потерь (регрессионные и ранжирующие) для моделей XGBoost и CatBoost. Настройте основные параметры моделей (глубина, кол-во деревьев, глубина, скорость обучения, регуляризация).  
Сравните построенные модели с точки зрения метрики nDCG на контроле и проанализируйте полученные результаты:
  - какая модель работает лучше всего для данной задачи? 
  - в чем достоинства/недостатки каждой? 
  - сравните модели между собой: 
   - получается ли сравнимое качество линейного pointwise подхода с остальными моделями? 
   - заметна ли разница в качестве при использовании бустинга с разными функциями потерь?

In [154]:
import warnings
warnings.filterwarnings('ignore')

In [220]:
useful_columns = ['query_id', 'window_size', 'relevance', 'rubrics_num', 'working_days_num', 'is_region_(2,)',
       'is_region_(143,)', 'is_region_(213,)', "is_region_code_('RU',)",
       "is_region_code_('TR',)", "is_region_code_('UA',)",
       'distance', 'word_count_query', 'word_count_org', 'is_same_region',
       'intersection_1_gramms', 'jaccard_1_gramms', 'intersection_2_gramms',
       'jaccard_2_gramms', 'intersection_3_gramms', 'jaccard_3_gramms', "is_query_('bg',)",
       "is_query_('mk',)", "is_query_('ru',)", "is_query_('tr',)",
       "is_org_name_('bg',)", "is_org_name_('mk',)", "is_org_name_('ru',)",
       "is_org_name_('tr',)", "is_org_name_('uk',)"]

In [194]:
def score_for_xgb(self, X, y):
    predict = self.predict(X)
    scores = []
    for query_id in X['query_id'].unique():
        mask = X['query_id'] == query_id

        if sum(mask) == 1:
            scores.append(1)
            continue

        score = ndcg_score(np.asarray([y[mask]]),
                       np.asarray([predict[mask]]))
        scores.append(score)
    return np.mean(scores)

In [195]:
def fit_xgboost(train_df: pd.DataFrame, model_class, objective, qid_column=None):
    params = {
        'max_depth': [2, 3, 4, 5, 6],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [25, 50, 75, 100],
        'lambda': [0, 1],
        'objective':[objective]
        }
    XGBoost = model_class()
    model = GridSearchCV(XGBoost, params, verbose=1, n_jobs=-1, scoring=score_for_xgb)
    if qid_column is not None:
        model.fit(train_df.drop(columns=['relevance']), train_df['relevance'], qid=qid_column)
    else:
        model.fit(train_df.drop(columns=['relevance']), train_df['relevance'])
    return model

In [196]:
model = fit_xgboost(train_df=train[useful_columns], model_class=xgb.XGBRegressor, objective='reg:squarederror')
score_for_xgb(model, test[useful_columns].drop(columns='relevance'), test['relevance'])

Fitting 5 folds for each of 120 candidates, totalling 600 fits


0.7787615582648851

In [197]:
model = fit_xgboost(train_df=train[useful_columns], model_class=xgb.XGBRanker, objective='rank:pairwise', qid_column=train['query_id'])
score_for_xgb(model, test[useful_columns].drop(columns='relevance'), test['relevance'])

Fitting 5 folds for each of 120 candidates, totalling 600 fits


0.7790807761739877

In [198]:
model = fit_xgboost(train_df=train[useful_columns], model_class=xgb.XGBRanker, objective='rank:ndcg', qid_column=train['query_id'])
score_for_xgb(model, test[useful_columns].drop(columns='relevance'), test['relevance'])

Fitting 5 folds for each of 120 candidates, totalling 600 fits


0.6651910238443524

In [233]:
train_pool = Pool(
    data=train[useful_columns].drop(columns=['relevance', 'query_id']).values,
    label=train['relevance'].values,
    group_id=train['query_id'].values
)

test_pool = Pool(
     data=test[useful_columns].drop(columns=['relevance', 'query_id']).values,
    label=test['relevance'].values,
    group_id=test['query_id'].values)

In [315]:
def fit_catboost(trial, loss_function):
    params = {
        'depth': trial.suggest_categorical('depth', [2, 3, 4, 5, 6]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1]),
        'n_estimators': trial.suggest_categorical('n_estimators', [25, 50, 75, 100]),
        'reg_lambda': trial.suggest_categorical('reg_lambda', [0, 1]),
        }

    catboost = CatBoostRanker(loss_function=loss_function, verbose=False, **params)

    catboost.fit(train_pool, eval_set=test_pool)

    return score_for_xgb(catboost, test[categorical_columns + useful_columns].drop(columns=['relevance']), test['relevance'])

In [272]:
test[useful_columns].drop(columns=['relevance', 'query_id'])

,window_size,rubrics_num,working_days_num,"is_region_(2,)","is_region_(143,)","is_region_(213,)","is_region_code_('RU',)","is_region_code_('TR',)","is_region_code_('UA',)",distance,...,jaccard_3_gramms,"is_query_('bg',)","is_query_('mk',)","is_query_('ru',)","is_query_('tr',)","is_org_name_('bg',)","is_org_name_('mk',)","is_org_name_('ru',)","is_org_name_('tr',)","is_org_name_('uk',)"
0,0.019706,1,7,0.0,0.0,0.0,0.0,0.0,1.0,0.413171,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
32,0.019706,1,7,0.0,0.0,0.0,0.0,0.0,1.0,0.390185,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
31,0.019706,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.509190,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
30,0.019706,1,7,0.0,0.0,0.0,0.0,0.0,1.0,0.606463,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
29,0.019706,4,7,0.0,0.0,0.0,0.0,0.0,1.0,0.757159,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8861,0.042153,2,7,1.0,0.0,0.0,1.0,0.0,0.0,0.024858,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8860,0.042153,1,7,0.0,0.0,0.0,1.0,0.0,0.0,828.119269,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8872,0.042153,2,5,0.0,0.0,0.0,1.0,0.0,0.0,25.256190,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8865,0.042153,2,6,1.0,0.0,0.0,1.0,0.0,0.0,0.229120,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [275]:
study = optuna.create_study(direction="maximize")
study.optimize(functools.partial(fit_catboost, loss_function='RMSE'), n_trials=50)
study.best_trial.value

0.7644994316070595

In [276]:
study = optuna.create_study(direction="maximize")
study.optimize(functools.partial(fit_catboost, loss_function='YetiRank'), n_trials=50)
study.best_trial.value

0.7609800092533126

In [263]:
study = optuna.create_study(direction="maximize")
study.optimize(functools.partial(fit_catboost, loss_function='YetiRankPairwise'), n_trials=50)
study.best_trial.value

0.7496550772875981

лучше все себя показал XGBoost c функцией pairwise

XGBoost себя в целом лучше показал чем CatBoost, хотя возможно это потому что я его подольше тюнил

регрессия почти также хоршо дает скор как и pairwise. с одной стороны регрессия лучше попадает когда ситуация какая-то вырожденная например если одна организация крутая а дркгие все фигня. с другой сторон регрессия решает более сложную задачу теми же методами, поэтому оптимизирует хуже

удивительно что 'rank:ndcg' для XGBoost совсем плохо работает хотя казалось бы

**(1 балл) Задание 3.** Одним из основных преимуществ CatBoost'a является обработка категориальных факторов «из коробки». Добавьте в датасет различные категориальные факторы из данных и обучите заново CatBoost модели. Улучшилось ли качество?

In [301]:
categorical_columns = ['region', 'org_id', 'org_region_id', 'region_code']
for column in categorical_columns:
    test[column] = test[column].astype("string")
    train[column] = train[column].astype("string")

In [313]:
train_pool = Pool(
    data=train[categorical_columns + useful_columns].drop(columns=['relevance']),
    cat_features=categorical_columns,
    label=train['relevance'],
    group_id=train['query_id']
)

test_pool = Pool(
     data=test[categorical_columns + useful_columns].drop(columns=['relevance']),
    cat_features=categorical_columns,
    label=test['relevance'],
    group_id=test['query_id'])

In [305]:
test[categorical_columns].dtypes

region           string
org_id           string
org_region_id    string
region_code      string
dtype: object

region           string
org_id           string
org_region_id    string
region_code      string
dtype: object

In [316]:
study = optuna.create_study(direction="maximize")
study.optimize(functools.partial(fit_catboost, loss_function='RMSE'), n_trials=50)
study.best_trial.value

0.7808536265644548

In [317]:
study = optuna.create_study(direction="maximize")
study.optimize(functools.partial(fit_catboost, loss_function='YetiRank'), n_trials=50)
study.best_trial.value

0.785022387001286

In [318]:
study = optuna.create_study(direction="maximize")
study.optimize(functools.partial(fit_catboost, loss_function='YetiRankPairwise'), n_trials=50)
study.best_trial.value

0.7829480674697985

о, куль
стало лучше
даже не ожидал как-то
все-таки не просто так там в яндексе сидят

**(2 балла) Задание 5.** Постройте любую нейросетевую ранжирующую модель
с помощью tensorflow_ranking или используйте dssm модель на tensorflow/keras/torch по аналогии с моделями на практическом занятии.